In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/10 02:44:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.createOrReplaceTempView('green')

In [4]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [5]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1, 2
ORDER BY 1, 2
""")

In [6]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000002|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [8]:
df_green_revenue.write.parquet('data/report/revenue/green', mode='overwrite')

In [9]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1, 2
""")

In [10]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [11]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [12]:
df_yellow.createOrReplaceTempView('yellow')

In [13]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [14]:
df_yellow_revenue = spark.sql("""
SELECT
    date_trunc('hour', tpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) as number_records
FROM yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1, 2
""")

In [15]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [16]:
!ls -lhR data/report/

data/report/:
total 4.0K
drwxr-xr-x 4 sweantu sweantu 4.0K Apr 10 02:46 revenue

data/report/revenue:
total 48K
-rw-r--r-- 1 sweantu sweantu    0 Apr  7 15:08 _SUCCESS
drwxr-xr-x 2 sweantu sweantu 4.0K Apr 10 02:45 green
-rw-r--r-- 1 sweantu sweantu  38K Apr  7 15:08 part-00000-17f8bb39-32c1-4d24-837a-f5be5e713008-c000.snappy.parquet
drwxr-xr-x 2 sweantu sweantu 4.0K Apr 10 02:46 yellow

data/report/revenue/green:
total 6.2M
-rw-r--r-- 1 sweantu sweantu    0 Apr 10 02:45 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 313K Apr 10 02:45 part-00000-fd2e133c-5d6d-4999-9c67-8b2230702166-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 312K Apr 10 02:45 part-00001-fd2e133c-5d6d-4999-9c67-8b2230702166-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 311K Apr 10 02:45 part-00002-fd2e133c-5d6d-4999-9c67-8b2230702166-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 312K Apr 10 02:45 part-00003-fd2e133c-5d6d-4999-9c67-8b2230702166-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 313K Apr 10 02:45 part-000

In [17]:
df_join = df_green_revenue.join(df_yellow_revenue, on=['hour', 'zone'], how='outer')

In [18]:
df_join.show()

+-------------------+----+------------------+--------------+------------------+--------------+
|               hour|zone|            amount|number_records|            amount|number_records|
+-------------------+----+------------------+--------------+------------------+--------------+
|2020-01-01 00:00:00|   3|              null|          null|              25.0|             1|
|2020-01-01 00:00:00|   4|              null|          null|1004.3000000000002|            57|
|2020-01-01 00:00:00|   7| 769.7299999999996|            45| 455.1700000000001|            38|
|2020-01-01 00:00:00|  12|              null|          null|             107.0|             6|
|2020-01-01 00:00:00|  37|            175.67|             6|161.60999999999999|             7|
|2020-01-01 00:00:00|  40|168.97999999999996|             8|             89.97|             5|
|2020-01-01 00:00:00|  45|              null|          null| 732.4800000000002|            42|
|2020-01-01 00:00:00|  47|              13.3|     

In [19]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')


In [20]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [21]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [22]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

In [23]:
df_join.repartition(200).write.parquet('data/report/revenue/total', mode='overwrite')

In [24]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [25]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [26]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [27]:
df_join.repartition(200).write.parquet('data/report/revenue/total', mode='overwrite')

In [28]:
df_join = spark.read.parquet('data/report/revenue/total')

In [29]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [30]:
df_zones = spark.read.parquet('zones')

In [31]:
df_zones

DataFrame[LocationID: string, Borough: string, Zone: string, service_zone: string]

In [32]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [34]:
df_result.drop('zone', 'LocationID').write.parquet('tmp/revenue-zones', mode='overwrite')

In [35]:
!ls -lh tmp/revenue-zones/

total 18M
-rw-r--r-- 1 sweantu sweantu    0 Apr 10 02:48 _SUCCESS
-rw-r--r-- 1 sweantu sweantu 2.7M Apr 10 02:48 part-00000-5e1c9825-69f0-4fe1-8034-adb80ab26e97-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 2.7M Apr 10 02:48 part-00001-5e1c9825-69f0-4fe1-8034-adb80ab26e97-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 2.7M Apr 10 02:48 part-00002-5e1c9825-69f0-4fe1-8034-adb80ab26e97-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 2.7M Apr 10 02:48 part-00003-5e1c9825-69f0-4fe1-8034-adb80ab26e97-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 2.7M Apr 10 02:48 part-00004-5e1c9825-69f0-4fe1-8034-adb80ab26e97-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 2.7M Apr 10 02:48 part-00005-5e1c9825-69f0-4fe1-8034-adb80ab26e97-c000.snappy.parquet
-rw-r--r-- 1 sweantu sweantu 1.3M Apr 10 02:48 part-00006-5e1c9825-69f0-4fe1-8034-adb80ab26e97-c000.snappy.parquet
